# The mosquito question

While hiking through the woods of Ontario several years ago, I became obsessed with this question: how many mosquitoes would it take to gang up and kill you?

Or, more specifically, how many mosquitoes would it take, constantly biting you as much as they can, to overcome your body's natural regeneration and threaten your life?

Other people out there have tried to answer this:
* [Outdoors.org](https://www.outdoors.org/resources/amc-outdoors/health-and-safety/how-many-mosquito-bites-would-kill-you/)
* [Business Insider](https://www.businessinsider.com/how-many-mosquitoes-drain-person-2016-2)

But I find the detail lacking, so we're going to approach this mathmatically using Pandas and a number of assumptions. This analysis is done for a lightning talk at NICAR22, a data journalism conference.

First, let's import pandas.

In [96]:
import pandas as pd

## The human variables

Here are the values we're using based on assumptions about human beings. These are all based on basic research online and all volumes are in millilitres. unless otherwise specified. Because red cells regenerate at a different rate than your plasma, we calculate them both separately.

Blood regeneration data is sourced from the [American Red Cross website](https://www.redcrossblood.org/faq.html). According to this source, it takes 4-6 weeks to regenerate the lost red blood cells from a donated pint of blood, and a day to regenerate the lost plasma from a donated pint of blood.

Note that while other attempts to answer this question deal with the amount of blood a human being has in its body, it's irrelevant here, as we're only interested in the tipping point where mosquitoes overwhelm your body's natural regeneration.

In [97]:
percent_blood_plasma = 0.55 ## Your blood is roughly 55% plasma. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss
percent_blood_redcells = 0.45 ## The non-plasma part of blood is made up of mostly red blood cells. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss

ml_per_pint = 473

redcell_daily_regen_fastest = ((1 * percent_blood_redcells) / 28) * ml_per_pint # Pints of pure RBC per day. We multiply by percent_blood_redcells to get the regen for a full pint of pure RBC.
redcell_daily_regen_slowest = ((1 * percent_blood_redcells) / 42) * ml_per_pint # This is the slowest that RBCs regenerate.
plasma_daily_regen = ((1 * percent_blood_plasma) / 1) * ml_per_pint  # Pints of pure plasma per day

## The mosquito variables

According to [the earlier-mentioned story by Business Insider](https://www.businessinsider.com/how-many-mosquitoes-drain-person-2016-2), a mosquito can take 5 microlitres (0.005 mL) per meal. The best estimate I could find on how often a single mosquito can bite you is every 3 days [here](https://mosquitoenemy.com/how-much-blood-can-a-mosquito-drink/).

When mosquitoes bite, they take a volume of blood that is made up of plasma and red blood cells (among other things), in a proportion that is roughly equal to the proportion of each in each human's blood (55% plasma, 45% red blood cells). Each of these blood components have different regeneration rates, so here, we calculate each separately to find out which would threaten a human life first.

In [98]:
daily_single_mosquito_drain = -0.005 / 3 # This is divided by three because a mosquito can take about 0.005 of blood every three days, at which point the digestion process is complete.

# RBC and plasma will be taken at slightly different rates in every bite.
single_mosquito_drain_rate_plasma = daily_single_mosquito_drain * percent_blood_plasma
single_mosquito_drain_rate_redcells = daily_single_mosquito_drain * percent_blood_redcells

## Building the swarm

Let's build out our swarm of mosquitoes: a table where each row represents the net blood drain of a certain number of mosquitoes.

In [99]:
## An empty Pandas dataframe.
df = pd.DataFrame()

# A column from 1 to 2 million, representing how many mosquitoes.
df["mosquitoes"] = range(1, 1000000)

df["net_redcell_regen_fastest"] = redcell_daily_regen_fastest + df['mosquitoes'] * single_mosquito_drain_rate_redcells 
df["net_redcell_regen_slowest"] = redcell_daily_regen_slowest + df['mosquitoes'] * single_mosquito_drain_rate_redcells 
df["net_plasma_regen"] = plasma_daily_regen + df['mosquitoes'] * single_mosquito_drain_rate_plasma

df = df.set_index("mosquitoes")

display(df)

,net_redcell_regen_fastest,net_redcell_regen_slowest,net_plasma_regen
mosquitoes,,,
1,7.601036,5.067107,260.149083
2,7.600286,5.066357,260.148167
3,7.599536,5.065607,260.147250
4,7.598786,5.064857,260.146333
5,7.598036,5.064107,260.145417
...,...,...,...
999995,-742.394464,-744.928393,-656.512083
999996,-742.395214,-744.929143,-656.513000
999997,-742.395964,-744.929893,-656.513917


Finally, we find the point in each column where the values flip from positive regeneration rate per day to a negative rate per day (the "danger zone").

In [100]:
danger_point_redcells_fastest = df.index[df['net_redcell_regen_fastest'] <= 0][0]
danger_point_redcells_slowest = df.index[df['net_redcell_regen_slowest'] <= 0][0]
danger_point_plasma = df.index[df['net_plasma_regen'] <= 0][0]

display("Red blood cell danger point (fastest regen): " + danger_point_redcells_lower.astype(str) + " mosquitoes.")
display("Red blood cell danger point (slowest regen): " + danger_point_redcells_upper.astype(str) + " mosquitoes.")
display("Blood plasma danger point: " + danger_point_plasma.astype(str) + " mosquitoes.")

'Red blood cell danger point (fastest regen): 10136 mosquitoes.'

'Red blood cell danger point (slowest regen): 6758 mosquitoes.'

'Blood plasma danger point: 283800 mosquitoes.'